In [2]:
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_pickle("data/20240813.gkg.first-1000-news.pkl")

In [4]:
gkg = pd.read_pickle("data/20240813.gkg.pkl")

In [5]:
list_urls = gkg["first_url"].head(100).to_list()

In [6]:
len(list_urls)

100

In [7]:
from newspaper import Article
from newspaper import Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
def get_article(url):
    try:
        article = Article(url, config=config)
        article.download()
        article.parse()
        article.nlp()
        data = {
            "title": article.title,
            "text": article.text,
            "summary": article.summary
        }
        return data
    except Exception as e:
        return {
            "title": "ERROR",
            "text": str(e),
            "summary": "ERROR"
        }

In [8]:
import time
def get_article(url):
    start_time = time.time()
    download_time = parse_time = nlp_time = None
    
    try:
        article = Article(url, config=config)
        
        start_download = time.time()
        article.download()
        download_time = time.time() - start_download
        
        start_parse = time.time()
        article.parse()
        parse_time = time.time() - start_parse
        
        start_nlp = time.time()
        article.nlp()
        nlp_time = time.time() - start_nlp
        
        data = {
            "title": article.title,
            "text": article.text,
            "summary": article.summary,
        }
    except Exception as e:
        data = {
            "title": "ERROR",
            "text": str(e),
            "summary": "ERROR",
        }
    finally:
        total_time = time.time() - start_time
        data = {
            **data,
            "download": download_time,
            "parse": parse_time,
            "nlp": nlp_time,
            "total": total_time
        }
        return data

In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def scrape_articles_parallel(urls, max_workers=10):
    results = [None] * len(urls)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_index = {executor.submit(get_article, url): i for i, url in enumerate(urls)}
        for future in as_completed(future_to_index):
            index = future_to_index[future]
            results[index] = future.result()
    return results

In [10]:
articles = scrape_articles_parallel(list_urls,10)

In [29]:
articles = scrape_articles_parallel(list_urls,20)

/Users/ahueteg/Documents/workspace/ahueteg/hackatons/factored-datathon-2024-ariteam/.venv/lib/python3.12/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [44]:

articles = scrape_articles_parallel(list_urls,40)

/Users/ahueteg/Documents/workspace/ahueteg/hackatons/factored-datathon-2024-ariteam/.venv/lib/python3.12/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/ahueteg/Documents/workspace/ahueteg/hackatons/factored-datathon-2024-ariteam/.venv/lib/python3.12/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [13]:
list_urls[30]

'https://www.portstephensexaminer.com.au/story/8728260/port-stephens-man-pleads-not-guilty-to-axe-handle-assault/?cs=30'

In [14]:
articles[30]

{'title': 'Man on bail denies bashing drunk person with axe handle',
 'text': "Raymond Terrace courthouse. File picture\n\nA MAN accused of bashing a drunk person with an axe handle at Port Stephens has formally denied the charge.\n\nSubscribe now for unlimited access .\n\nor signup to continue reading\n\nAll articles from our website\n\nThe digital version of Today's Paper\n\nBreaking news alerts direct to your inbox\n\nAll articles from the other in your area\n\nJarrod Tarrant did not appear in Raymond Terrace Local Court on Monday when his solicitor pleaded not guilty to assault occasioning actual bodily harm, on his behalf.\n\nThe 29-year-old was on bail on the night of July 26 when a man was punched, kicked, and beaten with an axe handle - suffering a ruptured spleen which required treatment in a hospital intensive care unit - at Shoal Bay.\n\nThe court heard when the matter was first mentioned three days after Mr Tarrant's arrest he planned to fight the charge on the basis he sai

In [15]:
import multiprocessing

def get_core_count():
    return multiprocessing.cpu_count()

print(f"Number of CPU cores: {get_core_count()}")

Number of CPU cores: 10


In [33]:
pd.DataFrame(articles).to_csv("data/20240813.gkg.first-1000-news-articles.csv", index=False)

In [30]:
pd.DataFrame(articles).tail()

,title,text,summary,download,parse,nlp,total
995,Soldier charged for Kiss driver's road death o...,"Laurel V Williams\n\nJALANI Mapp, the man char...","Laurel V Williams JALANI Mapp, the man charged...",8.456538,0.210626,0.002457,8.669670
996,Bangladesh Sees 12-Year High Inflation Amid Pr...,Bangladesh's inflation in July reached a 12-ye...,Bangladesh's inflation in July reached a 12-ye...,4.836724,1.499970,0.001541,6.338301
997,ERROR,Article `download()` failed with HTTPConnectio...,ERROR,10.675692,NaN,NaN,10.675741
998,Your superb shots of the Northern Lights visib...,The Aurora Borealis was visible in parts of th...,The Aurora Borealis was visible in parts of th...,9.594916,0.327279,0.001642,9.923888
999,LQR House Inc. Announces 382% Year-Over-Year R...,"MIAMI BEACH, FL / ACCESSWIRE / August 13, 2024...","In July 2023, LQR House reported revenues of $...",3.938475,25.957049,0.003842,29.899412


In [46]:
ar = pd.DataFrame(articles)

ar.sort_values("total", ascending=False).head(10)

,title,text,summary,download,parse,nlp,total
9998,"NITDA, Japan partner to empower startups with ...",The National Information Technology Developmen...,The National Information Technology Developmen...,4.829097,206.816207,0.002367,211.647736
7976,SVB&T Corp.: SVB&T Corporation Reports 2024 Se...,"SVB&T Corporation (OTCQX:SVBT), parent company...","Interest income increased $890,000 compared to...",5.704282,111.365592,0.013440,117.083366
8071,Franco-Nevada Reports Q2 2024 Results; New Min...,New Mine Start-ups and Acquisitions\n\n(in U.S...,Q2 2024 Portfolio Updates Precious Metal asset...,6.259472,96.307791,0.036811,102.604130
9428,Ponsse Oyj : Half year Report for 1 January 30...,Ponsse's Half-year Report for 1 January - 30 J...,DISTRIBUTION NETWORK In the new operating mode...,7.137853,88.686141,0.036498,95.860548
5760,1843 Seated Dollar,"Author Replies: 12 / Views: 1,690\n\nValued Me...",Pillar of the Community United States 4049 Pos...,5.871923,75.527072,0.003525,81.402565
6408,More than 20 injured in German train crash,BERLIN - An east German regional train struck ...,BERLIN - An east German regional train struck ...,8.011999,68.955060,0.001326,76.968432
7918,Syndicated Article – Your Fort Dodge,,,4.412640,72.175424,0.000181,76.588318
2269,Defunct N.S. regional development agency to un...,HALIFAX - A forensic audit will determine what...,"Percy Paris, the economic and rural developmen...",8.450237,67.218402,0.002903,75.671610
6542,CTV Montreal: Stephane Giroux at the courthouse,You are now being redirected to the BCE.ca web...,You are now being redirected to the BCE.ca web...,9.847173,65.702806,0.005332,75.555381
3720,Young people see online slurs as jokes: AP-MTV...,WASHINGTON - Is it ever OK to tweet that a gir...,"Jaded by the Internet free-for-all, teens and ...",8.151193,66.477123,0.006284,74.634714
